In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob
import os
import pandas as pd
from langdetect import detect

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [2]:
current_city = 'LosAngeles'

In [3]:
def read_and_concatenate_csv_files(city_name, directory='../data/cities'):
    pattern = os.path.join(directory, f'reviews_{city_name}_en*.csv.gz')
    files = glob.glob(pattern)
    
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        df_list.append(df)

    print(df_list)
    
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
    else:
        combined_df = pd.DataFrame()  # Return an empty DataFrame if no files found
    
    return combined_df

In [4]:
# multiple files
reviews = read_and_concatenate_csv_files(current_city)
print(reviews.head())

# single file
#reviews = pd.read_csv(f'../data/cities/reviews_{current_city}_en.csv.gz', compression='gzip')

[        listing_id         id        date  reviewer_id reviewer_name  \
0              109     449036  2011-08-15       927861         Edwin   
1              109   74506539  2016-05-15     22509885          Jenn   
2             2708   13994902  2014-06-09     10905424       Kuberan   
3             2708   14606598  2014-06-23      2247288       Camilla   
4             2708   39597339  2015-07-25     27974696        Fallon   
...            ...        ...         ...          ...           ...   
199995     5135865  459691215  2019-05-27    257772152        Ho Nam   
199996     5135865  471051391  2019-06-16    219675410         Artie   
199997     5135865  489974845  2019-07-17    274077763       Dominic   
199998     5135865  491862434  2019-07-20     91156789          Anar   
199999     4830835   29579642  2015-04-09     12550756        Andrea   

                                                 comments  
0       The host canceled my reservation the day befor...  
1       Me and

Ground_truth: review_scores_rating

## Preparing llm

In [5]:
model_name = "sohan-ai/sentiment-analysis-model-amazon-reviews"
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained(model_name)
model.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [19]:
# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Apply the function to the DataFrame
reviews['language'] = reviews['comments'].apply(detect_language)

# Filter for English texts
reviews['is_english'] = reviews['language'] == 'en'

In [27]:
def add_sentiment_features(row):
    global count
    count += 1
    if count % 1000 == 0:
        print(f'row: {count}')
    #if count > 4500:
    #    if count % 1 == 0:
    #        print(f'row: {count} | content: {row["comments"]}')
    text = row['comments']
    #if len(text) > 512:
    #    text = text[:512]
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    
    outputs = model(**inputs)
    logits = outputs.logits.detach().cpu().numpy()
    return logits[0][0], logits[0][1]

In [25]:
reviews = reviews[reviews['language'] == 'en']

In [26]:
reviews[reviews['language'] != 'en']

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,is_english


In [28]:
count = 0


reviews['comments'] = reviews['comments'].astype(str)
reviews['comments'] = reviews['comments'].apply(lambda x: x[:512] if len(x) > 512 else x)
reviews['feature_1'], reviews['feature_2'] = zip(*reviews.apply(add_sentiment_features, axis=1))

C:\Users\big10\AppData\Local\Temp\ipykernel_30672\2241461642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['comments'] = reviews['comments'].astype(str)
C:\Users\big10\AppData\Local\Temp\ipykernel_30672\2241461642.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['comments'] = reviews['comments'].apply(lambda x: x[:512] if len(x) > 512 else x)


row: 1000
row: 2000
row: 3000
row: 4000
row: 5000
row: 6000
row: 7000
row: 8000
row: 9000
row: 10000
row: 11000
row: 12000
row: 13000
row: 14000
row: 15000
row: 16000
row: 17000
row: 18000
row: 19000
row: 20000
row: 21000
row: 22000
row: 23000
row: 24000
row: 25000
row: 26000
row: 27000
row: 28000
row: 29000
row: 30000
row: 31000
row: 32000
row: 33000
row: 34000
row: 35000
row: 36000
row: 37000
row: 38000
row: 39000
row: 40000
row: 41000
row: 42000
row: 43000
row: 44000
row: 45000
row: 46000
row: 47000
row: 48000
row: 49000
row: 50000
row: 51000
row: 52000
row: 53000
row: 54000
row: 55000
row: 56000
row: 57000
row: 58000
row: 59000
row: 60000
row: 61000
row: 62000
row: 63000
row: 64000
row: 65000
row: 66000
row: 67000
row: 68000
row: 69000
row: 70000
row: 71000
row: 72000
row: 73000
row: 74000
row: 75000
row: 76000
row: 77000
row: 78000
row: 79000
row: 80000
row: 81000
row: 82000
row: 83000
row: 84000
row: 85000
row: 86000
row: 87000
row: 88000
row: 89000
row: 90000
row: 91000
row: 920

C:\Users\big10\AppData\Local\Temp\ipykernel_30672\2241461642.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['feature_1'], reviews['feature_2'] = zip(*reviews.apply(add_sentiment_features, axis=1))
C:\Users\big10\AppData\Local\Temp\ipykernel_30672\2241461642.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['feature_1'], reviews['feature_2'] = zip(*reviews.apply(add_sentiment_features, axis=1))


In [29]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,is_english,feature_1,feature_2
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...,en,True,2.184049,-2.119719
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...,en,True,-3.378882,3.399332
2,2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start ...,en,True,-2.092010,2.063507
3,2708,14606598,2014-06-23,2247288,Camilla,Charles is just amazing and he made my stay sp...,en,True,-2.447996,2.458068
4,2708,39597339,2015-07-25,27974696,Fallon,Staying with Chas was an absolute pleasure. He...,en,True,-2.002622,1.969444


In [35]:
#reviews.to_csv('LosAngeles_intermediat.csv')

In [37]:
X = reviews.drop(columns=['listing_id','id','date','reviewer_id','reviewer_name','comments','language','is_english'])

In [38]:
#X = X.drop(columns=['xgb_predict'])

In [39]:
X

,feature_1,feature_2
0,2.184049,-2.119719
1,-3.378882,3.399332
2,-2.092010,2.063507
3,-2.447996,2.458068
4,-2.002622,1.969444
...,...,...
1478433,-2.343008,2.356458
1478434,-3.010834,3.034031
1478435,-2.626419,2.677549
1478436,-2.428064,2.452944


In [40]:
X.head()

,feature_1,feature_2
0,2.184049,-2.119719
1,-3.378882,3.399332
2,-2.092010,2.063507
3,-2.447996,2.458068
4,-2.002622,1.969444


In [41]:
with open('XGBoost_kaggle_data_classifier.pkl', 'rb') as file:
    xgb_loaded = pickle.load(file)

# Use the loaded model to make predictions
reviews['xgb_predict'] = xgb_loaded.predict(X) + 1

C:\Users\big10\AppData\Local\Temp\ipykernel_30672\2285221612.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['xgb_predict'] = xgb_loaded.predict(X) + 1


In [42]:
with open('random_forest_kaggle_data_classifier.pkl', 'rb') as file:
    rf_loaded = pickle.load(file)

# Use the loaded model to make predictions
reviews['rf_predict'] = rf_loaded.predict(X) + 1

C:\Users\big10\AppData\Local\Temp\ipykernel_30672\2316187759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['rf_predict'] = rf_loaded.predict(X) + 1


In [44]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,is_english,feature_1,feature_2,xgb_predict,rf_predict
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...,en,True,2.184049,-2.119719,2,2
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...,en,True,-3.378882,3.399332,5,4
2,2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start ...,en,True,-2.092010,2.063507,5,5
3,2708,14606598,2014-06-23,2247288,Camilla,Charles is just amazing and he made my stay sp...,en,True,-2.447996,2.458068,5,5
4,2708,39597339,2015-07-25,27974696,Fallon,Staying with Chas was an absolute pleasure. He...,en,True,-2.002622,1.969444,5,5


In [45]:
reviews.to_csv(f'../data/cities/reviews_{current_city}_en_predictions_added.csv.gz', compression='gzip')

In [43]:
current_city

'LosAngeles'